In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

Calcula estatisticas e taxas de enchimento


Tempo: 2 horas

## utils

In [2]:
def q3(x):
    """
    Calcula o primeiro e terceiro quartil
    """
    return x.quantile(0.75)

def q1(x):
    return x.quantile(0.25)

In [3]:
def rolling_stats(df, window):
    """
    Calcula estatísticas deslizantes (rolling window) por grupo com base em colunas específicas.

    Para cada combinação de 'info_adicional', 'local_id' e 'tipo', aplica uma janela deslizante
    de `window` dias sobre as colunas 'delta_nivel', 'delta_tempo_horas' e 'taxa_enchimento_hora',
    após aplicar um shift (deslocamento) de 1 (este shift evita que use o proprio dia). Em seguida, calcula as estatisticas (média, mínimo, máximo, mediana, desvio padrão, Q1 e Q3).
    """
     
    # Ordena e define o índice
    df_sorted = df.sort_values('data')

    results = []
    for _, group in df_sorted.groupby(['info_adicional', 'local_id', 'tipo']):
        group = group.set_index('data').sort_index()
        group.index = pd.to_datetime(group.index)  # Garante que o índice seja datetime

        group_rolled = (
            group[['delta_nivel', 'delta_tempo_horas', 'taxa_enchimento_hora']]
            .shift(1)
            .rolling(f'{window}D', min_periods=1)
            .agg({
                'delta_nivel': ['mean', 'min', 'max', 'median', 'std', q1, q3],
                'delta_tempo_horas': ['mean', 'min', 'max', 'median', 'std', q1, q3],
                'taxa_enchimento_hora': ['mean', 'min', 'max', 'median', 'std', q1, q3]
            })
        )

        # Renomeia colunas
        group_rolled.columns = [
            f"{col[0]}_{col[1]}" if not isinstance(col[1], str) else f"{col[0]}_{col[1]}"
            for col in group_rolled.columns
        ]
        
        group_result = group_rolled.copy()
        group_result[['info_adicional', 'local_id', 'tipo']] = group[['info_adicional', 'local_id', 'tipo']]
        group_result = group_result.reset_index()
        results.append(group_result)

    return pd.concat(results, ignore_index=True)

In [4]:
def rolling_stats_nivel(df, window):
    """
    Calcula estatísticas deslizantes sobre o nível de enchimento, no momento da recolha.
    """
    
    # Filtra apenas recolha == 0
    df = df[df['recolha'] == 0]
    
    df_sorted = df.sort_values('data')

    results = []
    for _, group in df_sorted.groupby(['info_adicional', 'local_id', 'tipo']):
        group = group.set_index('data').sort_index()
        group.index = pd.to_datetime(group.index)  # Garante que o índice seja datetime

        # Rolling com agregações, incluindo quartis
        group_rolled = (
            group[['nivel']]
            .shift(1)
            .rolling(f'{window}D', min_periods=1)
            .agg(['mean', 'min', 'max', 'median', 'std', q1, q3])
        )

       # Renomeia colunas
        group_rolled.columns = [
            f"{col[0]}_{col[1]}" if not isinstance(col[1], str) else f"{col[0]}_{col[1]}"
            for col in group_rolled.columns
        ]
        # Adiciona colunas de identificação
        group_result = group_rolled.copy()
        group_result[['info_adicional', 'local_id', 'tipo']] = group[['info_adicional', 'local_id', 'tipo']]
        group_result = group_result.reset_index()
        results.append(group_result)

    return pd.concat(results, ignore_index=True)

In [5]:
def met_1(df):

    """
    Aplica um método de imputação e preparação de dados de nível para análise de enchimento.
    Esta função trata registos com nível igual a zero, substituindo-os por valores médios por
    combinação de ('local_id', 'tipo', 'cluster', 'info_adicional'). Em seguida, ordena e
    estrutura os dados para cálculo de variações (deltas) de nível e tempo, removendo
    inconsistências como deltas negativos.

    Etapas principais:
    - Separa registos com `nivel == 0`. (observacoes das recolhas)
    - Calcula a média do nível por grupo (das observacoes quando existe mais do que uma observacao para o mesmo tipo no mesmo cluster).
    - Insere uma linha representativa com a média para cada grupo.
    - Coloca primeiro as linhas onde o nivel é 0 e depois as observacoes para poder aferir qual o valor que foi recolhido (que será o proximo valor do mesmo tipo)
    - Ordena os dados, calcula a próxima medição de nível (`prox_nivel`), e aplica função de deltas.
    - Remove-se as linhas com nivel 0, pois apenas são uteis para saber qual valor foi recolhido e remove-se onde os deltas são negativos.
    """


    
    # Separar linhas com nivel == 0
    df_nivel_zero = df[df['nivel'] == 0].copy()
    
    # Linhas com nivel != 0
    df_nivel_positivo = df[df['nivel'] != 0].copy()

    # Calcular média por tipo e cluster, arredondada
    media_por_tipo_cluster = (
                                df_nivel_positivo
                                .groupby(['local_id', 'tipo', 'cluster', 'info_adicional'])['nivel']
                                .mean()
                                #.round()
                                #.astype(int)
                                .reset_index()
                            )

    # Obter uma linha representativa por tipo+cluster+local_id+info_adicional
    df_unico_por_tipo_cluster = (
                                    df_nivel_positivo.sort_values(by='time')
                                    .drop_duplicates(subset=['local_id', 'tipo', 'cluster', 'info_adicional'])
                                    .merge(media_por_tipo_cluster, on=['local_id', 'tipo', 'cluster', 'info_adicional'], suffixes=('', '_media'))
                                    .drop(columns=['nivel'])
                                    .rename(columns={'nivel_media': 'nivel'})
                            )

    # Concatenar com as linhas de nível 0
    df_resultado = pd.concat([df_nivel_zero, df_unico_por_tipo_cluster], ignore_index=True)
    
    # Ordenar: primeiro os de nível ≠ 0, depois os 0 (dentro de cada cluster)
    df_resultado['nivel_zero_flag'] = (df_resultado['nivel'] == 0).astype(int)
    df_resultado = df_resultado.sort_values(by=['cluster', 'nivel_zero_flag', 'timestamp', 'tipo']).reset_index(drop=True)
    df_resultado.drop(columns='nivel_zero_flag', inplace=True)

    # Adiciona a coluna 'prox_nivel' considerando o grupo correto
    df_resultado['recolha'] = (
                                df_resultado
                                .groupby(['tipo', 'local_id', 'info_adicional', 'cluster'])['nivel']
                                .shift(-1)
                             ).fillna(1)


    # Remover linhas com nível 
    df_resultado = df_resultado[df_resultado['nivel'] != 0].reset_index(drop=True)
    
    # Ordenar por tempo para cálculo dos deltas
    df_resultado = df_resultado.sort_values(by='timestamp').reset_index(drop=True)
    
    # Aplicar por grupo
    df_resultado = (
                        df_resultado
                        .groupby(['local_id', 'tipo', 'info_adicional'], group_keys=False)
                        .apply(calcular_deltas)
                    )
                    
    df_resultado = df_resultado.sort_values(by=['timestamp', 'info_adicional', 'local_id', 'tipo']).reset_index(drop=True)
    
    #Remove linhas onde o delta é negativo
    df_resultado = df_resultado[(df_resultado['delta_nivel'].isna()) | (df_resultado['delta_nivel'] >= 0)]    
    
    df_resultado.drop(columns='recolha_anterior', inplace=True)

    return df_resultado

In [6]:
# Função para aplicar por grupo
def calcular_deltas(grupo):
    """
    Calcula as diferenças (deltas) de nível e tempo entre observações consecutivas.
    Faz a recolha do nivel anterior, que será 0 se o ponto foi recolhido ou o valor observado se não foi recolhido

    """
  # Garante que o grupo está ordenado cronologicamente
    grupo = grupo.sort_values(by='timestamp').copy()
    
    # Armazena o nível anterior real para cada linha
    grupo['nivel_anterior_real'] = grupo['nivel'].shift(1)
    
    # Armazena a flag de recolha da linha anterior
    grupo['recolha_anterior'] = grupo['recolha'].shift(1)
    
    # Se a recolha anterior foi 0 (ou seja, houve recolha), o nível anterior é 0.
    # Caso contrário, usa o nível anterior real. Isso simula o reinício do contentor após recolha.
    grupo['nivel_anterior'] = np.where(grupo['recolha_anterior'] == 0, 0, grupo['nivel_anterior_real'])
    
    # Timestamp da linha anterior
    grupo['time_anterior'] = grupo['timestamp'].shift(1)
    
    # Diferença entre o nível atual e o anterior 
    grupo['delta_nivel'] = grupo['nivel'] - grupo['nivel_anterior']
    
    # Diferença de tempo entre observações consecutivas
    grupo['delta_tempo'] = grupo['timestamp'] - grupo['time_anterior']
    
    # Converte o delta de tempo para horas 
    grupo['delta_tempo_horas'] = grupo['delta_tempo'].dt.total_seconds() / 3600
    
    # Taxa de enchimento por hora = variação de nível / tempo passado
    grupo['taxa_enchimento_hora'] = grupo['delta_nivel'] / grupo['delta_tempo_horas']
    
    # Retorna o DataFrame com as novas colunas calculadas
    return grupo

In [7]:
def pre_process():
    """
    Realiza o pré-processamento dos dados de nível a partir de múltiplas fontes e aplica clusterização temporal, normaliza o nome dos circuitos
    e agrupa os eventos em clusters de 5 minutos (info_adicional e local_id)
    """

    
    df1 = pd.read_excel("../data/raw/seletiva_nivel.xlsx", sheet_name='Result 1')
    df2 = pd.read_excel("../data/raw/seletiva_nivel.xlsx", sheet_name='Result 2')

    seletiva_df = pd.concat([df1, df2], ignore_index=True)

    locais_path = "../data/raw/locais.xlsx"
    locais_df = pd.read_excel(locais_path)
        
    seletiva_df = seletiva_df.drop(columns=['produto_id'])
    seletiva_df = seletiva_df.drop(columns=['local_uuid_id'])#Remover as colunas que não interessam
    seletiva_df = seletiva_df[seletiva_df['local_id'] != 6394] #Este ponto é removido pois ao criar as rotas com o openrouteservice dá erro devido às
                                                               # suas coordenadas

    locais_df.rename(columns={'id': 'local_id'}, inplace=True)

    # Filtrar apenas os IDs comuns
    ids_comuns = set(locais_df['local_id']).intersection(set(seletiva_df['local_id']))
    seletiva_df = seletiva_df[seletiva_df['local_id'].isin(ids_comuns)]

    substituicoes = {
    'Circuito 03 32': 'Circuito 03',
    'Cricuito 08': 'Circuito 08',
    'Circ. Ilhas S': 'Circ. Ilhas',
    'circuito 05': 'Circuito 05',
    'Circ. Ilhas 04': 'Circ. Ilhas',
    'Circuito 09 92': 'Circuito 09',
    'Circuito Ilhas': 'Circ. Ilhas'
    }

    # Ignorar linhas com circuitos "via recolha" ou "GP 91"
    seletiva_df = seletiva_df[~seletiva_df['info_adicional'].str.lower().isin(['via recolha', 'gp 91'])]
    
    # Aplicar substituições
    seletiva_df['info_adicional'] = seletiva_df['info_adicional'].replace(substituicoes)

        
    seletiva_df['time'] = pd.to_datetime(seletiva_df['time'], errors='coerce')
    seletiva_df['data'] = seletiva_df['time'].dt.date

    #  Conversão da coluna 'time' para timestamp
    seletiva_df['timestamp'] = pd.to_datetime(seletiva_df['time'], format='mixed')

    #  Definição do threshold de 5 minutos
    threshold = pd.Timedelta(minutes=5)
    
    # Ordenação para garantir que .diff() funcione bem
    df = seletiva_df.sort_values(by=['info_adicional', 'local_id', 'timestamp']).reset_index(drop=True)

    #  Aplicação da clusterização por (info_adicional, local_id)
    df = df.groupby(['info_adicional', 'local_id'], group_keys=False).apply(cluster_sessions)

    # Substitui o timestamp por cluster_start (primeiro timestamp do cluster)
    df['timestamp'] = df.groupby(['info_adicional', 'local_id', 'cluster'])['timestamp'].transform('first')

    df.drop(columns='time_diff', inplace=True)
    return df

In [8]:
def cluster_sessions(group):
    """
    Agrupa eventos de um mesmo grupo (info_adicional e local_id) em clusters de 5 minutos.
    Um novo cluster é iniciado sempre que o intervalo entre dois eventos consecutivos
    for maior que 5 minutos.
    """
    threshold = pd.Timedelta(minutes=5)
    group = group.copy()
    group['time_diff'] = group['timestamp'].diff()
    group['cluster'] = (group['time_diff'] > threshold).cumsum()
    return group

In [9]:
def prep_merge_base():
    """
    Lê o ficheiro com as instancias e cria um df com as instancias presentes nesse ficheiro.
    Renomeia os circuitos para ficarem com os mesmos nomes do df_resultado
    """
    
    caminho = "../data/tabela_observacao_recolha.csv"
    df_pontos = pd.read_csv(caminho, delimiter=';')

    df_pontos=df_pontos[['Data', 'Circuito', 'Tipo', 'local_id']]

    df_merge_base = df_pontos.rename(columns={
    'Data': 'data',
    'Circuito': 'info_adicional',
    'Tipo': 'tipo'
    })

    substituicoes = {
    'Circ_Ilhas': 'Circ. Ilhas',
    'Circuito_01': 'Circuito 01',
    'Circuito_02': 'Circuito 02',
    'Circuito_03': 'Circuito 03',
    'Circuito_04': 'Circuito 04',
    'Circuito_05': 'Circuito 05',
    'Circuito_06': 'Circuito 06',
    'Circuito_07': 'Circuito 07',
    'Circuito_08': 'Circuito 08',
    'Circuito_09': 'Circuito 09',
    'Circuito_10': 'Circuito 10',
    'Circuito_11': 'Circuito 11',
    'Circuito_12': 'Circuito 12',
    }

    df_merge_base['info_adicional'] = df_merge_base['info_adicional'].replace(substituicoes)

    return df_merge_base

In [10]:
def prod_cartesian(df):
    """
    Gera um DataFrame que representa o produto cartesiano entre um conjunto de datas
    fixas e as combinações únicas de grupos definidas pelas colunas
    'info_adicional', 'local_id' e 'tipo' presentes no DataFrame de entrada.
    """
    
    # Datas de interesse
    all_dates = pd.date_range(start='2024-01-02', end='2024-12-31', freq='D')
    
    # Combinações únicas de grupos
    group_keys = df[['info_adicional', 'local_id', 'tipo']].drop_duplicates()
    
    # Produto cartesiano entre datas e grupos
    full_index = (
        group_keys.assign(key=1)
        .merge(pd.DataFrame({'data': all_dates, 'key': 1}), on='key')
        .drop(columns='key')
    )

    full_index['data'] = pd.to_datetime(full_index['data'])
    df['data'] = pd.to_datetime(df['data'])

    df_full = full_index.merge(df, on=['data', 'info_adicional', 'local_id', 'tipo'], how='left')
    return df_full
    

In [11]:
def calcular_estats(df):
    """
    Calcula estatísticas móveis (rolling statistics) para diferentes janelas de tempo
    (30 e 90 dias) sobre as colunas relacionadas a níveis e taxas, aplicando agregações
    como média, mínimo, máximo, mediana, desvio padrão e quartis (Q1 e Q3).

    A função também renomeia as colunas resultantes para facilitar a identificação
    das estatísticas por janela e tipo de métrica.
    """

    
    rolling_1m = rolling_stats(df, 30)
    rolling_3m = rolling_stats(df, 90)

    rolling_1m_nivel = rolling_stats_nivel(df, 30)
    rolling_3m_nivel = rolling_stats_nivel(df, 90)

        # Flatten colunas se ainda forem MultiIndex
    if isinstance(rolling_1m.columns, pd.MultiIndex):
        rolling_1m.columns = [
            f"{col[0]}_{col[1]}" if col[1] else col[0]
            for col in rolling_1m.columns
        ]
    # Flatten colunas se ainda forem MultiIndex
    if isinstance(rolling_3m.columns, pd.MultiIndex):
        rolling_3m.columns = [
            f"{col[0]}_{col[1]}" if col[1] else col[0]
            for col in rolling_3m.columns
        ]
    # Flatten colunas se ainda forem MultiIndex
    if isinstance(rolling_1m_nivel.columns, pd.MultiIndex):
        rolling_1m_nivel.columns = [
            f"{col[0]}_{col[1]}" if col[1] else col[0]
            for col in rolling_1m_nivel.columns
        ]
    # Flatten colunas se ainda forem MultiIndex
    if isinstance(rolling_3m_nivel.columns, pd.MultiIndex):
        rolling_3m_nivel.columns = [
            f"{col[0]}_{col[1]}" if col[1] else col[0]
            for col in rolling_3m_nivel.columns
        ]

    rename_dict_mes = {
        'delta_nivel_mean': 'dn_mean_1m',
        'delta_nivel_min': 'dn_min_1m',
        'delta_nivel_max': 'dn_max_1m',
        'delta_nivel_median': 'dn_median_1m',
        'delta_nivel_std': 'dn_std_1m',
        'delta_nivel_q1' : 'dn_1m_q1',
        'delta_nivel_q3' : 'dn_1m_q3',

    
        'delta_tempo_horas_mean': 'dt_mean_1m',
        'delta_tempo_horas_min': 'dt_min_1m',
        'delta_tempo_horas_max': 'dt_max_1m',
        'delta_tempo_horas_median': 'dt_median_1m',
        'delta_tempo_horas_std': 'dt_std_1m',
        'delta_tempo_horas_q1': 'dt_1m_q1',
        'delta_tempo_horas_q3': 'dt_1m_q3',


    
        'taxa_enchimento_hora_mean': 'te_mean_1m',
        'taxa_enchimento_hora_min': 'te_min_1m',
        'taxa_enchimento_hora_max': 'te_max_1m',
        'taxa_enchimento_hora_median': 'te_median_1m',
        'taxa_enchimento_hora_std': 'te_std_1m',
        'taxa_enchimento_hora_q1': 'te_1m_q1',
        'taxa_enchimento_hora_q3': 'te_1m_q3'

        
        }


    rename_dict_trimestral = {
        'delta_nivel_mean': 'dn_mean_3m',
        'delta_nivel_min': 'dn_min_3m',
        'delta_nivel_max': 'dn_max_3m',
        'delta_nivel_median': 'dn_median_3m',
        'delta_nivel_std': 'dn_std_3m',
        'delta_nivel_q1' : 'dn_3m_q1',
        'delta_nivel_q3' : 'dn_3m_q3',

    
        'delta_tempo_horas_mean': 'dt_mean_3m',
        'delta_tempo_horas_min': 'dt_min_3m',
        'delta_tempo_horas_max': 'dt_max_3m',
        'delta_tempo_horas_median': 'dt_median_3m',
        'delta_tempo_horas_std': 'dt_std_3m',
        'delta_tempo_horas_q1': 'dt_3m_q1',
        'delta_tempo_horas_q3': 'dt_3m_q3',
        
    
        'taxa_enchimento_hora_mean': 'te_mean_3m',
        'taxa_enchimento_hora_min': 'te_min_3m',
        'taxa_enchimento_hora_max': 'te_max_3m',
        'taxa_enchimento_hora_median': 'te_median_3m',
        'taxa_enchimento_hora_std': 'te_std_3m',
        'taxa_enchimento_hora_q1': 'te_3m_q1',
        'taxa_enchimento_hora_q3': 'te_3m_q3'



        
    }
    rename_dict_mes_nivel = {
    
        'nivel_mean' : 'rec_mean_1m',          
        'nivel_min' :   'rec_min_1m',      
        'nivel_max' : 'rec_max_1m',         
        'nivel_median' : 'rec_median_1m',          
        'nivel_std' : 'rec_std_1m',
        'nivel_q1' : 'rec_1m_q1',
        'nivel_q3' : 'rec_1m_q3'

    
    }
    
    rename_dict_trimestral_nivel = {
    
        'nivel_mean' : 'rec_mean_3m',          
        'nivel_min' :   'rec_min_3m',      
        'nivel_max' : 'rec_max_3m',         
        'nivel_median' : 'rec_median_3m',          
        'nivel_std' : 'rec_std_3m',
        'nivel_q1' : 'rec_3m_q1',
        'nivel_q3' : 'rec_3m_q3'
    
    }

    rolling_1m = rolling_1m.rename(columns=rename_dict_mes)
    rolling_3m = rolling_3m.rename(columns=rename_dict_trimestral)
    rolling_1m_nivel = rolling_1m_nivel.rename(columns=rename_dict_mes_nivel)
    rolling_3m_nivel = rolling_3m_nivel.rename(columns=rename_dict_trimestral_nivel)

    
    
    rolling_1m = rolling_1m.groupby(['data', 'info_adicional', 'tipo', 'local_id']).first().reset_index()
    rolling_3m = rolling_3m.groupby(['data', 'info_adicional', 'tipo', 'local_id']).first().reset_index()
    rolling_1m_nivel = rolling_1m_nivel.groupby(['data', 'info_adicional', 'tipo', 'local_id']).first().reset_index()
    rolling_3m_nivel = rolling_3m_nivel.groupby(['data', 'info_adicional', 'tipo', 'local_id']).first().reset_index()

    
        

    return rolling_1m, rolling_3m, rolling_1m_nivel, rolling_3m_nivel

In [12]:
def merge(df_merge_base, rolling_1m, rolling_3m, rolling_1m_nivel, rolling_3m_nivel):
    """
    Faz o merge do df das instancias com os dfs das estatisticas com base nas colunas
    ['data', 'info_adicional', 'local_id', 'tipo'], garantindo que as datas
    estão no formato datetime com o parâmetro dayfirst=True.
    Aplica também substituições específicas na coluna 'info_adicional'.
    """
    
    df_merge_base['data'] = pd.to_datetime(df_merge_base['data'], dayfirst=True)
    rolling_1m['data'] = pd.to_datetime(rolling_1m['data'], dayfirst=True)
    rolling_3m['data'] = pd.to_datetime(rolling_3m['data'], dayfirst=True)
    rolling_1m_nivel['data'] = pd.to_datetime(rolling_1m_nivel['data'], dayfirst=True)
    rolling_3m_nivel['data'] = pd.to_datetime(rolling_3m_nivel['data'], dayfirst=True)

    df_final = (
        df_merge_base
        .merge(rolling_1m, on=['data', 'info_adicional', 'local_id', 'tipo'], how='left')
        .merge(rolling_3m, on=['data', 'info_adicional', 'local_id', 'tipo'], how='left')
        .merge(rolling_1m_nivel, on=['data', 'info_adicional', 'local_id', 'tipo'], how='left')
        .merge(rolling_3m_nivel, on=['data', 'info_adicional', 'local_id', 'tipo'], how='left')
        )

    alterar = {
    'Circ. Ilhas': 'Circ_Ilhas',
    'Circuito 01': 'Circuito_01',
    'Circuito 02': 'Circuito_02',
    'Circuito 03': 'Circuito_03',
    'Circuito 04': 'Circuito_04',
    'Circuito 05': 'Circuito_05',
    'Circuito 06': 'Circuito_06',
    'Circuito 07': 'Circuito_07',
    'Circuito 08': 'Circuito_08',
    'Circuito 09': 'Circuito_09',
    'Circuito 10': 'Circuito_10',
    'Circuito 11': 'Circuito_11',
    'Circuito 12': 'Circuito_12',
    }


    df_final['info_adicional'] = df_final['info_adicional'].replace(alterar)
    
    return df_final

# --------------------

## Geral

In [13]:
df=pre_process()
df_resultado=met_1(df)

In [14]:
df_copia = df_resultado.copy()
df_merge_base=prep_merge_base()
#df_full=prod_cartesian(df_copia)

In [51]:
df_copia["data"] = pd.to_datetime(df_copia["data"]).dt.normalize()
df_merge_base["data"] = pd.to_datetime(df_merge_base["data"]).dt.normalize()

In [52]:
df_copia


,id,time,info_adicional,tipo,nivel,local_id,data,timestamp,cluster,recolha,nivel_anterior_real,nivel_anterior,time_anterior,delta_nivel,delta_tempo,delta_tempo_horas,taxa_enchimento_hora
0,5280464,2024-01-02 04:20:43+00:00,Circ. Ilhas,Embalagens,1.0,3863,2024-01-02,2024-01-02 04:20:43+00:00,0,1.0,NaN,NaN,NaT,NaN,NaT,NaN,NaN
1,5280465,2024-01-02 04:20:43+00:00,Circ. Ilhas,Vidro,1.0,3863,2024-01-02,2024-01-02 04:20:43+00:00,0,1.0,NaN,NaN,NaT,NaN,NaT,NaN,NaN
2,5280466,2024-01-02 04:20:49+00:00,Circ. Ilhas,Embalagens,1.0,3864,2024-01-02,2024-01-02 04:20:49+00:00,0,1.0,NaN,NaN,NaT,NaN,NaT,NaN,NaN
3,5280467,2024-01-02 04:20:49+00:00,Circ. Ilhas,Vidro,1.0,3864,2024-01-02,2024-01-02 04:20:49+00:00,0,1.0,NaN,NaN,NaT,NaN,NaT,NaN,NaN
4,5280469,2024-01-02 04:32:14+00:00,Circ. Ilhas,Embalagens,2.0,3844,2024-01-02,2024-01-02 04:32:14+00:00,0,1.0,NaN,NaN,NaT,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378417,6421094,2024-12-31 15:01:32+00:00,Circuito 12,Vidro,3.0,4294,2024-12-31,2024-12-31 15:01:32+00:00,183,1.0,1.0,1.0,2024-12-30 18:03:42+00:00,2.0,0 days 20:57:50,20.963889,0.095402
378418,6421097,2024-12-31 15:05:56+00:00,Circuito 12,Embalagens,5.0,4396,2024-12-31,2024-12-31 15:05:56+00:00,186,0.0,5.0,5.0,2024-12-30 18:08:15+00:00,0.0,0 days 20:57:41,20.961389,0.000000
378423,6421106,2024-12-31 15:11:15+00:00,Circuito 12,Papel,3.0,4296,2024-12-31,2024-12-31 15:11:15+00:00,186,1.0,5.0,0.0,2024-12-30 18:12:40+00:00,3.0,0 days 20:58:35,20.976389,0.143018
378424,6421105,2024-12-31 15:11:15+00:00,Circuito 12,Vidro,4.0,4296,2024-12-31,2024-12-31 15:11:15+00:00,186,1.0,1.0,1.0,2024-12-30 18:12:40+00:00,3.0,0 days 20:58:35,20.976389,0.143018


In [53]:
# Filtra apenas as linhas onde recolha == 0
df_filtrado = df_copia[df_copia["recolha"] == 0]

# Seleciona as colunas desejadas e remove duplicadas
df_unicos = df_filtrado[["data", "info_adicional", "tipo"]].drop_duplicates()

# Conta quantas linhas únicas existem
total_unicos = df_unicos.shape[0]

print(f"Número de combinações únicas com recolha == 0: {total_unicos}")


Número de combinações únicas com recolha == 0: 3632


In [56]:
df_unicos

,data,info_adicional,tipo
8,2024-01-02,Circ. Ilhas,Papel
14,2024-01-02,Circuito 07,Papel
27,2024-01-02,Circuito 09,Papel
50,2024-01-02,Circuito 10,Papel
55,2024-01-02,Circuito 08,Papel
...,...,...,...
377578,2024-12-31,Circuito 04,Embalagens
377592,2024-12-31,Circuito 06,Embalagens
377609,2024-12-31,Circuito 10,Embalagens
377640,2024-12-31,Circuito 12,Vidro


In [59]:
df_unicos_filtrado = df_unicos[df_unicos["data"] == "2024-01-02"]
df_unicos_filtrado

,data,info_adicional,tipo
8,2024-01-02,Circ. Ilhas,Papel
14,2024-01-02,Circuito 07,Papel
27,2024-01-02,Circuito 09,Papel
50,2024-01-02,Circuito 10,Papel
55,2024-01-02,Circuito 08,Papel
62,2024-01-02,Circuito 06,Papel
101,2024-01-02,Circuito 03,Embalagens
104,2024-01-02,Circuito 09,Embalagens
159,2024-01-02,Circuito 11,Embalagens
638,2024-01-02,Circuito 11,Papel


In [54]:
df_copia[["data", "info_adicional", "tipo"]].drop_duplicates().shape

(8509, 3)

In [55]:
df_merge_base[["data", "info_adicional", "tipo"]].drop_duplicates().shape

(4948, 3)

In [43]:
df_merge_base[["data", "info_adicional", "tipo"]].drop_duplicates()

,data,info_adicional,tipo
0,2024-02-01,Circ. Ilhas,Papel
42,2024-02-01,Circuito 03,Embalagens
107,2024-02-01,Circuito 05,Embalagens
180,2024-02-01,Circuito 06,Papel
253,2024-02-01,Circuito 07,Papel
...,...,...,...
361860,2024-12-31,Circuito 10,Embalagens
361936,2024-12-31,Circuito 10,Vidro
362012,2024-12-31,Circuito 11,Embalagens
362080,2024-12-31,Circuito 12,Embalagens


In [44]:
df_copia[["data", "info_adicional", "tipo"]].drop_duplicates()

,data,info_adicional,tipo
0,2024-01-02,Circ. Ilhas,Embalagens
1,2024-01-02,Circ. Ilhas,Vidro
5,2024-01-02,Circ. Ilhas,Papel
10,2024-01-02,Circuito 07,Embalagens
11,2024-01-02,Circuito 07,Papel
...,...,...,...
377622,2024-12-31,Circuito 12,Vidro
377635,2024-12-31,Circuito 06,Papel
377666,2024-12-31,Circuito 12,Papel
377669,2024-12-31,Circuito 02,Papel


In [45]:
# Conta o número de linhas únicas com base nas colunas desejadas
base_unicos = df_merge_base[["data", "info_adicional", "tipo"]].drop_duplicates().shape[0]
copia_unicos = df_copia[["data", "info_adicional", "tipo"]].drop_duplicates().shape[0]

print(f"Linhas únicas em df_merge_base: {base_unicos}")
print(f"Linhas únicas em df_copia: {copia_unicos}")


Linhas únicas em df_merge_base: 4948
Linhas únicas em df_copia: 8509


In [46]:
df_base_unique = df_merge_base[["data", "info_adicional", "tipo"]].drop_duplicates()
df_copia_unique = df_copia[["data", "info_adicional", "tipo"]].drop_duplicates()

# Faz o merge para encontrar diferenças
df_diferenca = df_copia_unique.merge(df_base_unique, 
                                     on=["data", "info_adicional", "tipo"], 
                                     how='left', 
                                     indicator=True)

# Filtra apenas os que estão só no df_copia
df_so_no_copia = df_diferenca[df_diferenca['_merge'] == 'left_only'].drop(columns=['_merge'])

print(f"Número de linhas únicas que estão apenas no df_copia: {df_so_no_copia.shape[0]}")
df_so_no_copia.head()


Número de linhas únicas que estão apenas no df_copia: 4611


,data,info_adicional,tipo
1,2024-01-02,Circ. Ilhas,Vidro
2,2024-01-02,Circ. Ilhas,Papel
7,2024-01-02,Circuito 09,Vidro
9,2024-01-02,Circuito 10,Embalagens
12,2024-01-02,Circuito 08,Embalagens


In [60]:
df_base_unique = df_merge_base[["data", "info_adicional", "tipo"]].drop_duplicates()
df_copia_unique = df_copia[["data", "info_adicional", "tipo"]].drop_duplicates()

# Agora, invertemos o merge: partimos do df_merge_base
df_diferenca_inversa = df_base_unique.merge(
    df_copia_unique,
    on=["data", "info_adicional", "tipo"],
    how='left',
    indicator=True
)

# Pegamos apenas as linhas que estão apenas no df_merge_base
df_so_no_base = df_diferenca_inversa[df_diferenca_inversa['_merge'] == 'left_only'].drop(columns=['_merge'])

print(f"Número de linhas únicas que estão apenas no df_merge_base: {df_so_no_base.shape[0]}")
df_so_no_base.head()


Número de linhas únicas que estão apenas no df_merge_base: 1050


,data,info_adicional,tipo
12,2024-03-01,Circ. Ilhas,Papel
26,2024-04-01,Circ. Ilhas,Embalagens
35,2024-04-01,Circuito 04,Embalagens
36,2024-04-01,Circuito 04,Vidro
42,2024-04-01,Circuito 07,Papel


In [25]:
rolling_1m, rolling_3m, rolling_1m_nivel, rolling_3m_nivel = calcular_estats(df_copia)

KeyboardInterrupt: 

In [ ]:
df_final=merge(df_merge_base, rolling_1m, rolling_3m, rolling_1m_nivel, rolling_3m_nivel)
df_final = df_final.rename(columns={'data': 'Data'})
df_final = df_final.rename(columns={'info_adicional': 'Circuito'})
df_final = df_final.rename(columns={'tipo': 'Tipo'})
df_final.to_csv("../data/taxas_e_estatisticas.csv", index=False)

C:\Users\simao\AppData\Local\Temp\ipykernel_22488\2605845511.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['info_adicional', 'local_id'], group_keys=False).apply(cluster_sessions)
C:\Users\simao\AppData\Local\Temp\ipykernel_22488\639862392.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_deltas)
